In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob 
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
benign = pd.read_csv("./dataset_benign.csv")
malware = pd.read_csv("./dataset_malware.csv")

data = pd.concat([benign, malware], ignore_index=True)

data = data.sample(frac=1, ignore_index=True)

data.head()


,Name,e_magic,e_cblp,e_cp,e_crlc,e_cparhdr,e_minalloc,e_maxalloc,e_ss,e_sp,...,SectionMaxChar,SectionMainChar,DirectoryEntryImport,DirectoryEntryImportSize,DirectoryEntryExport,ImageDirectoryEntryExport,ImageDirectoryEntryImport,ImageDirectoryEntryResource,ImageDirectoryEntryException,ImageDirectoryEntrySecurity
0,VirusShare_39816161187ba5f462fa50d06cc60d44,23117,144,3,0,4,0,65535,0,184,...,3221225536,0,4,115,4,1090416,1087740,1134592,0,0
1,VirusShare_829e319f36c00442b3f9152c1b7882d4,23117,80,2,0,4,15,65535,0,184,...,3221225536,0,14,384,0,0,450560,499712,0,0
2,VirusShare_3c25f515ce4f6113c78e7479dfbb1b64,23117,144,3,0,4,0,65535,0,184,...,3758096512,0,2,5,0,0,190816,188416,0,0
3,VirusShare_36b131e892d393620a44b9837d86c2a3,23117,144,3,0,4,0,65535,0,184,...,3221225536,0,1,106,0,0,31540,40960,0,0
4,VirusShare_05ae3a409bc4183ef689aeb360093719,23117,80,2,0,4,15,65535,0,184,...,3221225536,0,4,25,0,0,90112,98304,66048,0


In [ ]:
data.info()

In [ ]:
plt.figure(figsize=(8, 6))
ax=sns.countplot(data['Malware'])
ax.set_xticklabels(['Benign', 'Malware'])

In [ ]:
# #The target is Malware Column {0=Benign, 1=Malware}
X = data.drop(['Name', 'Malware'], axis = 1)
y = data['Malware']

X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91175 entries, 0 to 91174
Data columns (total 76 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   e_magic                       91175 non-null  int64  
 1   e_cblp                        91175 non-null  int64  
 2   e_cp                          91175 non-null  int64  
 3   e_crlc                        91175 non-null  int64  
 4   e_cparhdr                     91175 non-null  int64  
 5   e_minalloc                    91175 non-null  int64  
 6   e_maxalloc                    91175 non-null  int64  
 7   e_ss                          91175 non-null  int64  
 8   e_sp                          91175 non-null  int64  
 9   e_csum                        91175 non-null  int64  
 10  e_ip                          91175 non-null  int64  
 11  e_cs                          91175 non-null  int64  
 12  e_lfarlc                      91175 non-null  int64  
 13  e

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=101)


74641    1
1496     1
30034    1
41570    1
6711     1
Name: Malware, dtype: int64